# Postelection data analysis #

#### Nobody said it was going to be fun. But hey! Here we are. In this notebook we: ####
* build an ASCII file parser (more like a .txt parser)
* clean up Nassau data courtesy of Rita and TC
* clean up Suffolk data, courtest of BOE (not pleased about the ASCII format) [No Suffolk data here. All of it is in Excel].
* Put it all together so we have one file that has results for the local race as well as presidential

In [335]:
import pandas as pd
import numpy as np
import re

In [336]:
# Path to the .txt file which holds Suffolk data
file_path = 'Results and analysis/24GE_res.txt'

# Define a dictionary for Town Codes
town_codes = {
    "0": "Shelter Island",
    "1": "Brookhaven",
    "2": "Huntington",
    "3": "Islip",
    "4": "Babylon",
    "5": "Smithtown",
    "6": "Southampton",
    "7": "East Hampton",
    "8": "Southold",
    "9": "Riverhead"
}

In [337]:
# Initialize structures to hold parsed data
ed_data = []
candidates = []

# Open and read the .txt file
with open(file_path, 'r') as file:
    lines = file.readlines()

    for line in lines:
        record_type = line[4]  # The fifth character indicates record type

        if record_type == 'R':
            # Reset the candidate list for each new race
            candidates = []

        elif record_type == 'C':
            # Parse candidate information and add to list of candidates for the current race
            candidate_name = line[5:30].strip()
            party_code = line[30:33].strip()
            row_lever = line[41:44].strip()
            # Combine candidate name with party for a unique identifier
            candidates.append({
                'name': f"{candidate_name} ({party_code})",
                'row_lever': row_lever
            })

        elif record_type == 'E' and candidates:
            # Parse the ED record
            ed_number = line[6:9].strip()
            town_code = line[5:6]
            congressional_district = line[34:35].strip()
            senatorial_district = line[35:36].strip()
            assembly_district = line[36:38].strip()
            legislative_district = line[38:40].strip()

            # Format ED number to be zero-padded (e.g., '013' for ED 13)
            ed_number_padded = ed_number.zfill(3)
            precinct_name = f"{town_code}{ed_number_padded}"  # Create precinct_name as four-character string

            # Initialize a dictionary for each ED entry with location information
            ed_record = {
                'ED': ed_number,
                'Town Code': town_code,
                'Town': town_codes.get(town_code, "Unknown"),
                'precinct_name': precinct_name,
                'Congressional District': congressional_district,
                'Senatorial District': senatorial_district,
                'Assembly District': assembly_district,
                'Legislative District': legislative_district
            }

            # Calculate the starting position for votes in the E record
            vote_start_position = 52
            # Loop through each candidate and retrieve their vote count from the correct position
            for idx, candidate in enumerate(candidates):
                # Each vote block for a candidate is 4 characters long
                vote_start = vote_start_position + (idx * 4)
                vote_end = vote_start + 4
                candidate_votes = int(line[vote_start:vote_end].strip() or '0')

                # Add candidate vote to the ED record using candidate's name as the column
                ed_record[candidate['name']] = candidate_votes

            # Append structured ED record to the list
            ed_data.append(ed_record)

In [373]:
# Convert the parsed ED data to a DataFrame, aligning columns by candidate name across EDs
df_ed_votes = pd.DataFrame(ed_data)

# Save to a CSV file
df_ed_votes.to_csv('Results and analysis/suffolk_output.csv', index=False)

print(f"Data saved to {'Results and analysis/suffolk_output.csv'}")

Data saved to Results and analysis/suffolk_output.csv


In [339]:
df_ed_votes

,ED,Town Code,Town,precinct_name,Congressional District,Senatorial District,Assembly District,Legislative District,"Harris, Kamala D (DEM)","Harris, Kamala D (WOR)",...,"Shaw, Lisa (IAC)","Damato, Paul L (REP)","Damato, Paul L (CON)","Scheuring, Maria C (DEM)","Zollo, John B (REP)","Zollo, John B (CON)","Grossman, Adam B (DEM)","Grossman, Adam B (CON)","Sartain, Karen M (DEM)","Sartain, Karen M (CON)"
0,001,0,Shelter Island,0001,1,1,1,1,239.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002,0,Shelter Island,0002,1,1,1,1,272.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,003,0,Shelter Island,0003,1,1,1,1,261.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,004,0,Shelter Island,0004,1,1,1,1,283.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,001,1,Brookhaven,1001,1,1,4,5,395.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9385,038,6,Southampton,6038,1,1,1,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,189.0,88.0,185.0,80.0
9386,039,6,Southampton,6039,1,1,1,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,324.0,117.0,326.0,108.0
9387,040,6,Southampton,6040,1,1,1,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,140.0,96.0,139.0,83.0
9388,041,6,Southampton,6041,1,1,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,182.0,68.0,191.0,64.0


## Part 2

Using the ASCII file TC McCarthy parsed with his instant-built [parser](https://github.com/newsdaycom/ASC2CSV/tree/main), we are going to look for some local results.

In [340]:
gen = pd.read_csv('Map data/GEN2024_preliminary.csv')

/var/folders/3w/v4769sl91ks57hsf6mhcg5_wglchws/T/ipykernel_33658/597256256.py:1: DtypeWarning: Columns (6,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  gen = pd.read_csv('Map data/GEN2024_preliminary.csv')


In [341]:
gen.head(6)

,contest_num,candidate_num,precinct_code,reg_voter_count,ballots_cast,party_code,district_type_id,district_code,contest_title,candidate_name,precinct_name,district_name,votes_allowed,referendum_flag,cr_code,lf_code
0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,113001.0,NaN,0.0,NaN,NaN,NaN
1,1.0,1.0,2.0,0.0,0.0,NaN,NaN,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,113002.0,NaN,0.0,NaN,NaN,NaN
2,1.0,1.0,3.0,0.0,0.0,NaN,NaN,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,113003.0,NaN,0.0,NaN,NaN,NaN
3,1.0,1.0,4.0,0.0,0.0,NaN,NaN,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,113004.0,NaN,0.0,NaN,NaN,NaN
4,1.0,1.0,5.0,0.0,0.0,NaN,NaN,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,113005.0,NaN,0.0,NaN,NaN,NaN
5,1.0,1.0,6.0,0.0,0.0,NaN,NaN,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,113006.0,NaN,0.0,NaN,NaN,NaN


In [342]:
# Convert precinct_name to text

gen = gen.dropna(subset=['precinct_name'])
gen['precinct_name'] = gen['precinct_name'].astype(int).astype(str)
gen

,contest_num,candidate_num,precinct_code,reg_voter_count,ballots_cast,party_code,district_type_id,district_code,contest_title,candidate_name,precinct_name,district_name,votes_allowed,referendum_flag,cr_code,lf_code
0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,113001,NaN,0.0,NaN,NaN,NaN
1,1.0,1.0,2.0,0.0,0.0,NaN,NaN,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,113002,NaN,0.0,NaN,NaN,NaN
2,1.0,1.0,3.0,0.0,0.0,NaN,NaN,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,113003,NaN,0.0,NaN,NaN,NaN
3,1.0,1.0,4.0,0.0,0.0,NaN,NaN,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,113004,NaN,0.0,NaN,NaN,NaN
4,1.0,1.0,5.0,0.0,0.0,NaN,NaN,NaN,REGISTERED VOTERS - TOTAL,REGISTERED VOTERS - TOTAL,113005,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106403,131.0,4.0,1076.0,1.0,1.0,NaN,NaN,NaN,"Proposal Number One, An Amendment",UNDER VOTES,517042,NaN,1.0,NaN,NaN,NaN
106404,131.0,4.0,1077.0,1.0,1.0,NaN,NaN,NaN,"Proposal Number One, An Amendment",UNDER VOTES,517043,NaN,1.0,NaN,NaN,NaN
106405,131.0,4.0,1078.0,18.0,18.0,NaN,NaN,NaN,"Proposal Number One, An Amendment",UNDER VOTES,517044,NaN,1.0,NaN,NaN,NaN
106406,131.0,4.0,1079.0,20.0,20.0,NaN,NaN,NaN,"Proposal Number One, An Amendment",UNDER VOTES,517045,NaN,1.0,NaN,NaN,NaN


In [343]:
gen[gen['candidate_name'].isin(['OVER VOTES','UNDER VOTES','WRITE-IN'])]['ballots_cast'].sum()

1805285.0

In [344]:
# Drop the over votes, under votes and write-ins.
# Unofficial results don't include them

gen = gen[~gen['candidate_name'].isin(['OVER VOTES','UNDER VOTES','WRITE-IN'])]

In [345]:
gen[gen['contest_title']=='BALLOTS CAST - TOTAL']

,contest_num,candidate_num,precinct_code,reg_voter_count,ballots_cast,party_code,district_type_id,district_code,contest_title,candidate_name,precinct_name,district_name,votes_allowed,referendum_flag,cr_code,lf_code
1080,2.0,1.0,1.0,867.0,867.0,NaN,NaN,NaN,BALLOTS CAST - TOTAL,BALLOTS CAST - TOTAL,113001,NaN,0.0,NaN,NaN,NaN
1081,2.0,1.0,2.0,657.0,657.0,NaN,NaN,NaN,BALLOTS CAST - TOTAL,BALLOTS CAST - TOTAL,113002,NaN,0.0,NaN,NaN,NaN
1082,2.0,1.0,3.0,460.0,460.0,NaN,NaN,NaN,BALLOTS CAST - TOTAL,BALLOTS CAST - TOTAL,113003,NaN,0.0,NaN,NaN,NaN
1083,2.0,1.0,4.0,796.0,796.0,NaN,NaN,NaN,BALLOTS CAST - TOTAL,BALLOTS CAST - TOTAL,113004,NaN,0.0,NaN,NaN,NaN
1084,2.0,1.0,5.0,582.0,582.0,NaN,NaN,NaN,BALLOTS CAST - TOTAL,BALLOTS CAST - TOTAL,113005,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2.0,1.0,1076.0,1.0,1.0,NaN,NaN,NaN,BALLOTS CAST - TOTAL,BALLOTS CAST - TOTAL,517042,NaN,0.0,NaN,NaN,NaN
2156,2.0,1.0,1077.0,34.0,34.0,NaN,NaN,NaN,BALLOTS CAST - TOTAL,BALLOTS CAST - TOTAL,517043,NaN,0.0,NaN,NaN,NaN
2157,2.0,1.0,1078.0,348.0,348.0,NaN,NaN,NaN,BALLOTS CAST - TOTAL,BALLOTS CAST - TOTAL,517044,NaN,0.0,NaN,NaN,NaN
2158,2.0,1.0,1079.0,295.0,295.0,NaN,NaN,NaN,BALLOTS CAST - TOTAL,BALLOTS CAST - TOTAL,517045,NaN,0.0,NaN,NaN,NaN


In [346]:
gen[gen['contest_title']=='BALLOTS CAST - TOTAL'].to_csv('ballots_total.csv')

In [347]:
# Drop the races we ain't interested in

gen = gen[~gen['contest_title'].isin(['BALLOTS CAST - BLANK','BALLOTS CAST - TOTAL','County Court Judge','District Court Judge District 4','Family Court Judge','Justice of the Supreme Court','Proposal Number One, An Amendment','REGISTERED VOTERS - TOTAL'])]

In [348]:
gen['total'] = gen.groupby(['precinct_name','contest_title'])['ballots_cast'].transform('sum')
gen

,contest_num,candidate_num,precinct_code,reg_voter_count,ballots_cast,party_code,district_type_id,district_code,contest_title,candidate_name,precinct_name,district_name,votes_allowed,referendum_flag,cr_code,lf_code,total
3240,101.0,1.0,1.0,387.0,387.0,DEM,NaN,NaN,Presidential Electors for President and Vice P...,Kamala D. Harris,113001,NaN,1.0,NaN,NaN,NaN,847.0
3241,101.0,1.0,2.0,333.0,333.0,DEM,NaN,NaN,Presidential Electors for President and Vice P...,Kamala D. Harris,113002,NaN,1.0,NaN,NaN,NaN,642.0
3242,101.0,1.0,3.0,240.0,240.0,DEM,NaN,NaN,Presidential Electors for President and Vice P...,Kamala D. Harris,113003,NaN,1.0,NaN,NaN,NaN,451.0
3243,101.0,1.0,4.0,346.0,346.0,DEM,NaN,NaN,Presidential Electors for President and Vice P...,Kamala D. Harris,113004,NaN,1.0,NaN,NaN,NaN,789.0
3244,101.0,1.0,5.0,275.0,275.0,DEM,NaN,NaN,Presidential Electors for President and Vice P...,Kamala D. Harris,113005,NaN,1.0,NaN,NaN,NaN,573.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102011,130.0,3.0,634.0,24.0,24.0,CON,CTC,TLB,City Judge City of Long Beach,Corey E. Klein,320020,City of Long Beach,1.0,NaN,NaN,NaN,517.0
102012,130.0,3.0,635.0,30.0,30.0,CON,CTC,TLB,City Judge City of Long Beach,Corey E. Klein,320021,City of Long Beach,1.0,NaN,NaN,NaN,640.0
102013,130.0,3.0,636.0,25.0,25.0,CON,CTC,TLB,City Judge City of Long Beach,Corey E. Klein,320022,City of Long Beach,1.0,NaN,NaN,NaN,728.0
102014,130.0,3.0,637.0,17.0,17.0,CON,CTC,TLB,City Judge City of Long Beach,Corey E. Klein,320023,City of Long Beach,1.0,NaN,NaN,NaN,516.0


In [349]:
gen_ed = gen.groupby(['precinct_name','contest_title','candidate_name'])['ballots_cast'].sum().reset_index()
gen_ed.head(5)

,precinct_name,contest_title,candidate_name,ballots_cast
0,113001,Member of Assembly District 13,Charles D. Lavine,427.0
1,113001,Member of Assembly District 13,Ruka Anzai,357.0
2,113001,Presidential Electors for President and Vice P...,Donald J. Trump,443.0
3,113001,Presidential Electors for President and Vice P...,Kamala D. Harris,404.0
4,113001,Representative in Congress District 3,Michael J. LiPetri Jr.,332.0


In [350]:
grouped = gen.groupby(['precinct_name','contest_title'])['ballots_cast'].sum().reset_index()
grouped.head(5)

,precinct_name,contest_title,ballots_cast
0,113001,Member of Assembly District 13,784.0
1,113001,Presidential Electors for President and Vice P...,847.0
2,113001,Representative in Congress District 3,825.0
3,113001,State Senator District 7,785.0
4,113001,United States Senator,817.0


In [351]:
gen_ed.head(10)

,precinct_name,contest_title,candidate_name,ballots_cast
0,113001,Member of Assembly District 13,Charles D. Lavine,427.0
1,113001,Member of Assembly District 13,Ruka Anzai,357.0
2,113001,Presidential Electors for President and Vice P...,Donald J. Trump,443.0
3,113001,Presidential Electors for President and Vice P...,Kamala D. Harris,404.0
4,113001,Representative in Congress District 3,Michael J. LiPetri Jr.,332.0
5,113001,Representative in Congress District 3,Thomas R. Suozzi,493.0
6,113001,State Senator District 7,Jack M. Martins,406.0
7,113001,State Senator District 7,Kim Keiserman,379.0
8,113001,United States Senator,Diane Sare,1.0
9,113001,United States Senator,Kirsten E. Gillibrand,412.0


In [352]:
gen_ed = gen_ed.merge(grouped, on=['precinct_name', 'contest_title'], suffixes=('', '_total'))
gen_ed.head(5)

,precinct_name,contest_title,candidate_name,ballots_cast,ballots_cast_total
0,113001,Member of Assembly District 13,Charles D. Lavine,427.0,784.0
1,113001,Member of Assembly District 13,Ruka Anzai,357.0,784.0
2,113001,Presidential Electors for President and Vice P...,Donald J. Trump,443.0,847.0
3,113001,Presidential Electors for President and Vice P...,Kamala D. Harris,404.0,847.0
4,113001,Representative in Congress District 3,Michael J. LiPetri Jr.,332.0,825.0


In [353]:
gen_ed['percentage'] = (gen_ed['ballots_cast'] / gen_ed['ballots_cast_total']) * 100
gen_ed.head(10)

,precinct_name,contest_title,candidate_name,ballots_cast,ballots_cast_total,percentage
0,113001,Member of Assembly District 13,Charles D. Lavine,427.0,784.0,54.464286
1,113001,Member of Assembly District 13,Ruka Anzai,357.0,784.0,45.535714
2,113001,Presidential Electors for President and Vice P...,Donald J. Trump,443.0,847.0,52.302243
3,113001,Presidential Electors for President and Vice P...,Kamala D. Harris,404.0,847.0,47.697757
4,113001,Representative in Congress District 3,Michael J. LiPetri Jr.,332.0,825.0,40.242424
5,113001,Representative in Congress District 3,Thomas R. Suozzi,493.0,825.0,59.757576
6,113001,State Senator District 7,Jack M. Martins,406.0,785.0,51.719745
7,113001,State Senator District 7,Kim Keiserman,379.0,785.0,48.280255
8,113001,United States Senator,Diane Sare,1.0,817.0,0.122399
9,113001,United States Senator,Kirsten E. Gillibrand,412.0,817.0,50.428397


In [354]:
gen_ed[gen_ed['percentage'].isna()]

,precinct_name,contest_title,candidate_name,ballots_cast,ballots_cast_total,percentage
3891,218073,Member of Assembly District 18,Danielle Samantha Smikle,0.0,0.0,NaN
3892,218073,Member of Assembly District 18,Noah Burroughs,0.0,0.0,NaN
3893,218073,Presidential Electors for President and Vice P...,Donald J. Trump,0.0,0.0,NaN
3894,218073,Presidential Electors for President and Vice P...,Kamala D. Harris,0.0,0.0,NaN
3895,218073,Representative in Congress District 4,Anthony P. D'Esposito,0.0,0.0,NaN
...,...,...,...,...,...,...
12235,510701,State Senator District 5,Lisa Lin,0.0,0.0,NaN
12236,510701,State Senator District 5,Steven D. Rhoads,0.0,0.0,NaN
12237,510701,United States Senator,Diane Sare,0.0,0.0,NaN
12238,510701,United States Senator,Kirsten E. Gillibrand,0.0,0.0,NaN


In [355]:
gen_ed.to_csv('gen_ed.csv')

In [356]:
gen_ed[gen_ed['precinct_name']=='510701']

,precinct_name,contest_title,candidate_name,ballots_cast,ballots_cast_total,percentage
12229,510701,Member of Assembly District 10,Aamir Sultan,0.0,0.0,NaN
12230,510701,Member of Assembly District 10,Steve Stern,0.0,0.0,NaN
12231,510701,Presidential Electors for President and Vice P...,Donald J. Trump,0.0,0.0,NaN
12232,510701,Presidential Electors for President and Vice P...,Kamala D. Harris,0.0,0.0,NaN
12233,510701,Representative in Congress District 3,Michael J. LiPetri Jr.,0.0,0.0,NaN
12234,510701,Representative in Congress District 3,Thomas R. Suozzi,0.0,0.0,NaN
12235,510701,State Senator District 5,Lisa Lin,0.0,0.0,NaN
12236,510701,State Senator District 5,Steven D. Rhoads,0.0,0.0,NaN
12237,510701,United States Senator,Diane Sare,0.0,0.0,NaN
12238,510701,United States Senator,Kirsten E. Gillibrand,0.0,0.0,NaN


In [357]:
# Write a function to create a new column
# 'category' which tells us what kinda race it is

def extract_category(contest_title):
    if 'Assembly District' in contest_title:
        return 'Assembly District'
    elif 'Congress District' in contest_title:
        return 'Congressional District'
    elif 'State Senator District' in contest_title:
        return 'State Senate District'
    elif 'Presidential' in contest_title:
        return 'President'
    elif 'United States Senator' in contest_title:
        return 'US Senator'
    else:
        return 'Other'

In [358]:
gen_ed['category'] = gen_ed['contest_title'].apply(extract_category)
gen_ed.tail(10)

,precinct_name,contest_title,candidate_name,ballots_cast,ballots_cast_total,percentage,category
14584,517045,Member of Assembly District 17,John K. Mikulin,154.0,274.0,56.204380,Assembly District
14585,517045,Presidential Electors for President and Vice P...,Donald J. Trump,152.0,286.0,53.146853,President
14586,517045,Presidential Electors for President and Vice P...,Kamala D. Harris,134.0,286.0,46.853147,President
14587,517045,Representative in Congress District 3,Michael J. LiPetri Jr.,146.0,282.0,51.773050,Congressional District
14588,517045,Representative in Congress District 3,Thomas R. Suozzi,136.0,282.0,48.226950,Congressional District
14589,517045,State Senator District 8,Alexis Weik,149.0,272.0,54.779412,State Senate District
14590,517045,State Senator District 8,Francis R. Dolan,123.0,272.0,45.220588,State Senate District
14591,517045,United States Senator,Diane Sare,0.0,285.0,0.000000,US Senator
14592,517045,United States Senator,Kirsten E. Gillibrand,138.0,285.0,48.421053,US Senator
14593,517045,United States Senator,Michael D. Sapraicone,147.0,285.0,51.578947,US Senator


In [359]:
# Checking if ballots_total adds up

grouped_df = gen_ed.groupby(['precinct_name', 'contest_title'], as_index=False).agg({
    'ballots_cast': 'sum',
    'ballots_cast_total': 'first'  # Keep the first occurrence of ballots_cast_total for comparison
})

# Check for any discrepancies
grouped_df['total_matches'] = grouped_df['ballots_cast'] == grouped_df['ballots_cast_total']

# Display rows where ballots_cast does not match ballots_cast_total
discrepancies = grouped_df[~grouped_df['total_matches']]

# Print summary
if discrepancies.empty:
    print("All ballots_cast totals match ballots_cast_total for each precinct and contest.")
else:
    print("Discrepancies found:")
    print(discrepancies[['precinct', 'contest', 'ballots_cast', 'ballots_cast_total']])

All ballots_cast totals match ballots_cast_total for each precinct and contest.


In [360]:
# Function to determine the President winner for each precinct, handling ties as specified

def determine_winner(group):
    group = group.dropna(subset=['percentage'])  # Skip rows with NaN percentages
    if not group.empty:
        max_percentage = group['percentage'].max()
        winners = group[group['percentage'] == max_percentage]  # Select all rows with the maximum percentage

        # Check if there is a tie (more than one candidate has the max percentage)
        if len(winners) > 1:
            # In the case of a tie, return specified values
            return pd.Series(['Tie', 50, winners['ballots_cast'].iloc[1], winners['ballots_cast_total'].max()], 
                             index=['race_winner', 'race_winning_perc', 'race_win_votes', 'ballots_cast_total'])
        else:
            # No tie, only one winner
            winner_row = winners.iloc[0]
            return pd.Series([winner_row['candidate_name'], max_percentage, winner_row['ballots_cast'], winner_row['ballots_cast_total']], 
                             index=['race_winner', 'race_winning_perc', 'race_win_votes', 'ballots_cast_total'])
    return pd.Series([None, None, None, None], index=['race_winner', 'race_winning_perc', 'race_win_votes', 'ballots_cast_total'])


In [361]:
# Apply the function to each precinct and create new columns 'President_winner' and 'winning_perc'
winners = gen_ed.groupby(['precinct_name','contest_title','category']).apply(determine_winner).reset_index()
winners

,precinct_name,contest_title,category,race_winner,race_winning_perc,race_win_votes,ballots_cast_total
0,113001,Member of Assembly District 13,Assembly District,Charles D. Lavine,54.464286,427.0,784.0
1,113001,Presidential Electors for President and Vice P...,President,Donald J. Trump,52.302243,443.0,847.0
2,113001,Representative in Congress District 3,Congressional District,Thomas R. Suozzi,59.757576,493.0,825.0
3,113001,State Senator District 7,State Senate District,Jack M. Martins,51.719745,406.0,785.0
4,113001,United States Senator,US Senator,Kirsten E. Gillibrand,50.428397,412.0,817.0
...,...,...,...,...,...,...,...
6144,517045,Member of Assembly District 17,Assembly District,John K. Mikulin,56.204380,154.0,274.0
6145,517045,Presidential Electors for President and Vice P...,President,Donald J. Trump,53.146853,152.0,286.0
6146,517045,Representative in Congress District 3,Congressional District,Michael J. LiPetri Jr.,51.773050,146.0,282.0
6147,517045,State Senator District 8,State Senate District,Alexis Weik,54.779412,149.0,272.0


In [362]:
gen_ed.to_csv('local_ed.csv')

In [363]:
# Function to get the President winner for each precinct

def get_prez_winner(group):
    prez_winner_row = group.loc[group['category'] == 'President']
    if not prez_winner_row.empty:
        prez_winner = prez_winner_row['race_winner'].values[0]
        prez_winning_perc = prez_winner_row['race_winning_perc'].values[0]
        prez_votes = prez_winner_row['race_win_votes'].values[0]
        prez_total_votes = prez_winner_row['ballots_cast_total'].values[0]
        return pd.Series([prez_winner, prez_winning_perc, prez_votes, prez_total_votes], index=['prez_winner', 'prez_winning_perc','prez_votes','prez_total_votes'])
    return pd.Series([None, None, None, None], index=['prez_winner', 'prez_winning_perc','prez_votes','prez_total_votes'])

In [364]:
# Apply the function to each precinct and create a new column 'prez_winner'

prez_winners = winners.groupby('precinct_name').apply(get_prez_winner).reset_index()
prez_winners

,precinct_name,prez_winner,prez_winning_perc,prez_votes,prez_total_votes
0,113001,Donald J. Trump,52.302243,443.0,847.0
1,113002,Kamala D. Harris,53.582555,344.0,642.0
2,113003,Kamala D. Harris,53.658537,242.0,451.0
3,113004,Donald J. Trump,54.499366,430.0,789.0
4,113005,Donald J. Trump,50.087260,287.0,573.0
...,...,...,...,...,...
1075,517041,Donald J. Trump,62.500000,210.0,336.0
1076,517042,Donald J. Trump,100.000000,1.0,1.0
1077,517043,Donald J. Trump,67.647059,23.0,34.0
1078,517044,Donald J. Trump,72.383721,249.0,344.0


In [365]:
winners = winners.merge(prez_winners, on='precinct_name', how='left')
winners

,precinct_name,contest_title,category,race_winner,race_winning_perc,race_win_votes,ballots_cast_total,prez_winner,prez_winning_perc,prez_votes,prez_total_votes
0,113001,Member of Assembly District 13,Assembly District,Charles D. Lavine,54.464286,427.0,784.0,Donald J. Trump,52.302243,443.0,847.0
1,113001,Presidential Electors for President and Vice P...,President,Donald J. Trump,52.302243,443.0,847.0,Donald J. Trump,52.302243,443.0,847.0
2,113001,Representative in Congress District 3,Congressional District,Thomas R. Suozzi,59.757576,493.0,825.0,Donald J. Trump,52.302243,443.0,847.0
3,113001,State Senator District 7,State Senate District,Jack M. Martins,51.719745,406.0,785.0,Donald J. Trump,52.302243,443.0,847.0
4,113001,United States Senator,US Senator,Kirsten E. Gillibrand,50.428397,412.0,817.0,Donald J. Trump,52.302243,443.0,847.0
...,...,...,...,...,...,...,...,...,...,...,...
6144,517045,Member of Assembly District 17,Assembly District,John K. Mikulin,56.204380,154.0,274.0,Donald J. Trump,53.146853,152.0,286.0
6145,517045,Presidential Electors for President and Vice P...,President,Donald J. Trump,53.146853,152.0,286.0,Donald J. Trump,53.146853,152.0,286.0
6146,517045,Representative in Congress District 3,Congressional District,Michael J. LiPetri Jr.,51.773050,146.0,282.0,Donald J. Trump,53.146853,152.0,286.0
6147,517045,State Senator District 8,State Senate District,Alexis Weik,54.779412,149.0,272.0,Donald J. Trump,53.146853,152.0,286.0


In [366]:
winners.head(15)

,precinct_name,contest_title,category,race_winner,race_winning_perc,race_win_votes,ballots_cast_total,prez_winner,prez_winning_perc,prez_votes,prez_total_votes
0,113001,Member of Assembly District 13,Assembly District,Charles D. Lavine,54.464286,427.0,784.0,Donald J. Trump,52.302243,443.0,847.0
1,113001,Presidential Electors for President and Vice P...,President,Donald J. Trump,52.302243,443.0,847.0,Donald J. Trump,52.302243,443.0,847.0
2,113001,Representative in Congress District 3,Congressional District,Thomas R. Suozzi,59.757576,493.0,825.0,Donald J. Trump,52.302243,443.0,847.0
3,113001,State Senator District 7,State Senate District,Jack M. Martins,51.719745,406.0,785.0,Donald J. Trump,52.302243,443.0,847.0
4,113001,United States Senator,US Senator,Kirsten E. Gillibrand,50.428397,412.0,817.0,Donald J. Trump,52.302243,443.0,847.0
5,113002,Member of Assembly District 13,Assembly District,Charles D. Lavine,57.382550,342.0,596.0,Kamala D. Harris,53.582555,344.0,642.0
6,113002,Presidential Electors for President and Vice P...,President,Kamala D. Harris,53.582555,344.0,642.0,Kamala D. Harris,53.582555,344.0,642.0
7,113002,Representative in Congress District 3,Congressional District,Thomas R. Suozzi,61.538462,384.0,624.0,Kamala D. Harris,53.582555,344.0,642.0
8,113002,State Senator District 7,State Senate District,Kim Keiserman,51.505017,308.0,598.0,Kamala D. Harris,53.582555,344.0,642.0
9,113002,United States Senator,US Senator,Kirsten E. Gillibrand,53.697749,334.0,622.0,Kamala D. Harris,53.582555,344.0,642.0


In [367]:
winners.groupby('precinct_name')['prez_total_votes'].sum()

precinct_name
113001    4235.0
113002    3210.0
113003    2255.0
113004    3945.0
113005    2865.0
           ...  
517041    1680.0
517042       5.0
517043     170.0
517044    1720.0
517045    1430.0
Name: prez_total_votes, Length: 1080, dtype: float64

In [368]:
# winners = winners[winners['category'] != 'President']
winners

,precinct_name,contest_title,category,race_winner,race_winning_perc,race_win_votes,ballots_cast_total,prez_winner,prez_winning_perc,prez_votes,prez_total_votes
0,113001,Member of Assembly District 13,Assembly District,Charles D. Lavine,54.464286,427.0,784.0,Donald J. Trump,52.302243,443.0,847.0
1,113001,Presidential Electors for President and Vice P...,President,Donald J. Trump,52.302243,443.0,847.0,Donald J. Trump,52.302243,443.0,847.0
2,113001,Representative in Congress District 3,Congressional District,Thomas R. Suozzi,59.757576,493.0,825.0,Donald J. Trump,52.302243,443.0,847.0
3,113001,State Senator District 7,State Senate District,Jack M. Martins,51.719745,406.0,785.0,Donald J. Trump,52.302243,443.0,847.0
4,113001,United States Senator,US Senator,Kirsten E. Gillibrand,50.428397,412.0,817.0,Donald J. Trump,52.302243,443.0,847.0
...,...,...,...,...,...,...,...,...,...,...,...
6144,517045,Member of Assembly District 17,Assembly District,John K. Mikulin,56.204380,154.0,274.0,Donald J. Trump,53.146853,152.0,286.0
6145,517045,Presidential Electors for President and Vice P...,President,Donald J. Trump,53.146853,152.0,286.0,Donald J. Trump,53.146853,152.0,286.0
6146,517045,Representative in Congress District 3,Congressional District,Michael J. LiPetri Jr.,51.773050,146.0,282.0,Donald J. Trump,53.146853,152.0,286.0
6147,517045,State Senator District 8,State Senate District,Alexis Weik,54.779412,149.0,272.0,Donald J. Trump,53.146853,152.0,286.0


In [369]:
winners['losing_prez_votes'] = winners['prez_total_votes'] - winners['prez_votes']
winners.head(16)

,precinct_name,contest_title,category,race_winner,race_winning_perc,race_win_votes,ballots_cast_total,prez_winner,prez_winning_perc,prez_votes,prez_total_votes,losing_prez_votes
0,113001,Member of Assembly District 13,Assembly District,Charles D. Lavine,54.464286,427.0,784.0,Donald J. Trump,52.302243,443.0,847.0,404.0
1,113001,Presidential Electors for President and Vice P...,President,Donald J. Trump,52.302243,443.0,847.0,Donald J. Trump,52.302243,443.0,847.0,404.0
2,113001,Representative in Congress District 3,Congressional District,Thomas R. Suozzi,59.757576,493.0,825.0,Donald J. Trump,52.302243,443.0,847.0,404.0
3,113001,State Senator District 7,State Senate District,Jack M. Martins,51.719745,406.0,785.0,Donald J. Trump,52.302243,443.0,847.0,404.0
4,113001,United States Senator,US Senator,Kirsten E. Gillibrand,50.428397,412.0,817.0,Donald J. Trump,52.302243,443.0,847.0,404.0
5,113002,Member of Assembly District 13,Assembly District,Charles D. Lavine,57.382550,342.0,596.0,Kamala D. Harris,53.582555,344.0,642.0,298.0
6,113002,Presidential Electors for President and Vice P...,President,Kamala D. Harris,53.582555,344.0,642.0,Kamala D. Harris,53.582555,344.0,642.0,298.0
7,113002,Representative in Congress District 3,Congressional District,Thomas R. Suozzi,61.538462,384.0,624.0,Kamala D. Harris,53.582555,344.0,642.0,298.0
8,113002,State Senator District 7,State Senate District,Kim Keiserman,51.505017,308.0,598.0,Kamala D. Harris,53.582555,344.0,642.0,298.0
9,113002,United States Senator,US Senator,Kirsten E. Gillibrand,53.697749,334.0,622.0,Kamala D. Harris,53.582555,344.0,642.0,298.0


In [370]:
winners.to_csv('nassau_race_winners.csv')

**Post analysis:**
Simultaneously, as I analyzed TC's data, I realized it only included Nassau. The nightmare was real. Only data for parts of assembly districts and congressional seats in Nassau. Suffolk's data was not just in ASCII, it was in PDF format. I remember feeding ChatGPT the coded PDF as well as the results layout. It spouted out one .csv with separate columns. I cannot remember now if it was decoded or if I used excel functions to do that (most likely the latter). All these files sit on my Newsday desktop Data -> Election 2024 -> Results and analysis. I will also leave a backup on my karnam.news Drive. 

In [372]:
# We are going to try a code that automates
# the calculation of percentage winning by district
# Let's begin with CD4

district = winners[winners['contest_title']=='Representative in Congress District 4']
total_prez = district['prez_total_votes'].sum()
votes_harris = district[district['prez_winner']=='Kamala D. Harris']
votes_trump = district[district['prez_winner']=='Donald J. Trump']
votes_tie = district[district['prez_winner']=='Tie']
total_harris = votes_harris['prez_votes'].sum() + votes_trump['losing_prez_votes'].sum() + votes_tie['prez_votes'].sum()
total_trump = votes_trump['prez_votes'].sum() + votes_harris['losing_prez_votes'].sum() + votes_tie['prez_votes'].sum()
perc_harris = total_harris/total_prez * 100
perc_trump = total_trump/total_prez * 100

print("Total sum of all prezident votes in CD-4:", total_prez)
# print("Votes won by Hariss in CD3:", votes_harris)
# print("Votes by Hariss in AD-20:", total_harris)
# print("Votes by Trump in AD-20:", total_trump)
print(f"Harris %: {perc_harris:.2f}, Trump %: {perc_trump:.2f}")

Total sum of all prezident votes in CD-4: 372655.0
Harris %: 50.55, Trump %: 49.45
